In [1]:
!git clone https://github.com/ultralytics/yolov5
!cd yolov5;pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 9322, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 9322 (delta 35), reused 14 (delta 0), pack-reused 9249
Receiving objects: 100% (9322/9322), 9.89 MiB | 24.81 MiB/s, done.
Resolving deltas: 100% (6457/6457), done.
     |████████████████████████████████| 636 kB 5.4 MB/s 


In [2]:
!git clone https://github.com/Shenggan/BCCD_Dataset.git
!git clone https://github.com/yukkyo/voc2coco.git

Cloning into 'BCCD_Dataset'...
remote: Enumerating objects: 800, done.
remote: Total 800 (delta 0), reused 0 (delta 0), pack-reused 800
Receiving objects: 100% (800/800), 7.39 MiB | 21.31 MiB/s, done.
Resolving deltas: 100% (378/378), done.
Cloning into 'voc2coco'...
remote: Enumerating objects: 423, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 423 (delta 3), reused 8 (delta 3), pack-reused 409
Receiving objects: 100% (423/423), 214.64 KiB | 2.13 MiB/s, done.
Resolving deltas: 100% (379/379), done.


In [3]:
import os
 
with open('/content/BCCD_Dataset/BCCD/labels.txt', "w") as f:
    f.write("WBC\n")
    f.write("RBC\n")
    f.write("Platelets\n")

!cat /content/BCCD_Dataset/BCCD/labels.txt

WBC
RBC
Platelets


In [4]:
%cd voc2coco
!python voc2coco.py --ann_dir /content/BCCD_Dataset/BCCD/Annotations \
    --ann_ids /content/BCCD_Dataset/BCCD/ImageSets/Main/train.txt \
    --labels /content/BCCD_Dataset/BCCD/labels.txt \
    --output /content/BCCD_Dataset/BCCD/train.json \
    --ext xml

!python voc2coco.py --ann_dir /content/BCCD_Dataset/BCCD/Annotations \
    --ann_ids /content/BCCD_Dataset/BCCD/ImageSets/Main/val.txt \
    --labels /content/BCCD_Dataset/BCCD/labels.txt \
    --output /content/BCCD_Dataset/BCCD/val.json \
    --ext xml

!python voc2coco.py --ann_dir /content/BCCD_Dataset/BCCD/Annotations \
    --ann_ids /content/BCCD_Dataset/BCCD/ImageSets/Main/test.txt \
    --labels /content/BCCD_Dataset/BCCD/labels.txt \
    --output /content/BCCD_Dataset/BCCD/test.json \
    --ext xml

/content/voc2coco
Start converting !
100% 205/205 [00:00<00:00, 4235.15it/s]
Start converting !
100% 87/87 [00:00<00:00, 3452.79it/s]
Start converting !
100% 72/72 [00:00<00:00, 3450.17it/s]


In [5]:
%cd /content

/content


In [6]:
!mkdir /content/bccd;
!cd /content/bccd; mkdir images; mkdir labels;
!cd /content/bccd/images; mkdir train; mkdir val; mkdir test;
!cd /content/bccd/labels; mkdir train; mkdir val; mkdir test;

In [7]:
# https://github.com/alexmihalyk23/COCO2YOLO.git
import json
import os
import shutil

class COCO2YOLO:
  # 소스 이미지 디렉토리와 Json annotation 파일, 타겟 이미지 디렉토리, 타겟 annotation 디렉토리를 생성자로 입력 받음. 
  def __init__(self, src_img_dir, json_file, tgt_img_dir, tgt_anno_dir):
    self.json_file = json_file
    self.src_img_dir = src_img_dir
    self.tgt_img_dir = tgt_img_dir
    self.tgt_anno_dir = tgt_anno_dir
    # json 파일과 타겟 디렉토리가 존재하는지 확인하고, 디렉토리의 경우는 없으면 생성. 
    self._check_file_and_dir(json_file, tgt_img_dir, tgt_anno_dir)
    # json 파일을 메모리로 로딩. 
    self.labels = json.load(open(json_file, 'r', encoding='utf-8'))
    # category id와 이름을 매핑하지만, 실제 class id는 이를 적용하지 않고 별도 적용. 
    self.coco_id_name_map = self._categories()
    self.coco_name_list = list(self.coco_id_name_map.values())
    print("total images", len(self.labels['images']))
    print("total categories", len(self.labels['categories']))
    print("total labels", len(self.labels['annotations']))
  
  # json 파일과 타겟 디렉토리가 존재하는지 확인하고, 디렉토리의 경우는 없으면 생성. 
  def _check_file_and_dir(self, file_path, tgt_img_dir, tgt_anno_dir):
    if not os.path.exists(file_path):
        raise ValueError("file not found")
    if not os.path.exists(tgt_img_dir):
        os.makedirs(tgt_img_dir)
    if not os.path.exists(tgt_anno_dir):
        os.makedirs(tgt_anno_dir)

  # category id와 이름을 매핑하지만, 추후에 class 명만 활용. 
  def _categories(self):
    categories = {}
    for cls in self.labels['categories']:
        categories[cls['id']] = cls['name']
    return categories
  
  # annotation에서 모든 image의 파일명(절대 경로 아님)과 width, height 정보 저장. 
  def _load_images_info(self):
    images_info = {}
    for image in self.labels['images']:
        id = image['id']
        file_name = image['file_name']
        if file_name.find('\\') > -1:
            file_name = file_name[file_name.index('\\')+1:]
        w = image['width']
        h = image['height']
  
        images_info[id] = (file_name, w, h)

    return images_info

  # ms-coco의 bbox annotation은 yolo format으로 변환. 좌상단 x, y좌표, width, height 기반을 정규화된 center x,y 와 width, height로 변환. 
  def _bbox_2_yolo(self, bbox, img_w, img_h):
    # ms-coco는 좌상단 x, y좌표, width, height
    x, y, w, h = bbox[0], bbox[1], bbox[2], bbox[3]
    # center x좌표는 좌상단 x좌표에서 width의 절반을 더함. center y좌표는 좌상단 y좌표에서 height의 절반을 더함.  
    centerx = bbox[0] + w / 2
    centery = bbox[1] + h / 2
    # centerx, centery, width, height를 이미지의 width/height로 정규화. 
    dw = 1 / img_w
    dh = 1 / img_h
    centerx *= dw
    w *= dw
    centery *= dh
    h *= dh
    return centerx, centery, w, h
  
  # image와 annotation 정보를 기반으로 image명과 yolo annotation 정보 가공. 
  # 개별 image당 하나의 annotation 정보를 가지도록 변환. 
  def _convert_anno(self, images_info):
    anno_dict = dict()
    for anno in self.labels['annotations']:
      bbox = anno['bbox']
      image_id = anno['image_id']
      category_id = anno['category_id']

      image_info = images_info.get(image_id)
      image_name = image_info[0]
      img_w = image_info[1]
      img_h = image_info[2]
      yolo_box = self._bbox_2_yolo(bbox, img_w, img_h)

      anno_info = (image_name, category_id, yolo_box)
      anno_infos = anno_dict.get(image_id)
      if not anno_infos:
        anno_dict[image_id] = [anno_info]
      else:
        anno_infos.append(anno_info)
        anno_dict[image_id] = anno_infos
    return anno_dict

  # class 명을 파일로 저장하는 로직. 사용하지 않음. 
  def save_classes(self):
    sorted_classes = list(map(lambda x: x['name'], sorted(self.labels['categories'], key=lambda x: x['id'])))
    print('coco names', sorted_classes)
    with open('coco.names', 'w', encoding='utf-8') as f:
      for cls in sorted_classes:
          f.write(cls + '\n')
    f.close()
  # _convert_anno(images_info)로 만들어진 anno 정보를 개별 yolo anno txt 파일로 생성하는 로직. 
  # coco2yolo()에서 anno_dict = self._convert_anno(images_info)로 만들어진 anno_dict를 _save_txt()에 입력하여 파일 생성
  def _save_txt(self, anno_dict):
    # 개별 image별로 소스 image는 타겟이미지 디렉토리로 복사하고, 개별 annotation을 타겟 anno 디렉토리로 생성. 
    for k, v in anno_dict.items():
      # 소스와 타겟 파일의 절대 경로 생성. 
      src_img_filename = os.path.join(self.src_img_dir, v[0][0])
      tgt_anno_filename = os.path.join(self.tgt_anno_dir,v[0][0].split(".")[0] + ".txt")
      #print('source image filename:', src_img_filename, 'target anno filename:', tgt_anno_filename)
      # 이미지 파일의 경우 타겟 디렉토리로 단순 복사. 
      shutil.copy(src_img_filename, self.tgt_img_dir)
      # 타겟 annotation 출력 파일명으로 classid, bbox 좌표를 object 별로 생성. 
      with open(tgt_anno_filename, 'w', encoding='utf-8') as f:
        #print(k, v)
        # 여러개의 object 별로 classid와 bbox 좌표를 생성. 
        for obj in v:
          cat_name = self.coco_id_name_map.get(obj[1])
          # category_id는 class 명에 따라 0부터 순차적으로 부여. 
          category_id = self.coco_name_list.index(cat_name)
          #print('cat_name:', cat_name, 'category_id:', category_id)
          box = ['{:.6f}'.format(x) for x in obj[2]]
          box = ' '.join(box)
          line = str(category_id) + ' ' + box
          f.write(line + '\n')

  # ms-coco를 yolo format으로 변환. 
  def coco2yolo(self):
    print("loading image info...")
    images_info = self._load_images_info()
    print("loading done, total images", len(images_info))

    print("start converting...")
    anno_dict = self._convert_anno(images_info)
    print("converting done, total labels", len(anno_dict))

    print("saving txt file...")
    self._save_txt(anno_dict)
    print("saving done")

  

In [8]:
# train 용 yolo 데이터 세트 생성. 
train_yolo_converter = COCO2YOLO(src_img_dir='/content/BCCD_Dataset/BCCD/JPEGImages', json_file='/content/BCCD_Dataset/BCCD/train.json',
                                 tgt_img_dir='/content/bccd/images/train', tgt_anno_dir='/content/bccd/labels/train')
train_yolo_converter.coco2yolo()

# val 용 yolo 데이터 세트 생성. 
val_yolo_converter = COCO2YOLO(src_img_dir='/content/BCCD_Dataset/BCCD/JPEGImages', json_file='/content/BCCD_Dataset/BCCD/val.json',
                                 tgt_img_dir='/content/bccd/images/val', tgt_anno_dir='/content/bccd/labels/val')
val_yolo_converter.coco2yolo()

# test 용 yolo 데이터 세트 생성. 
test_yolo_converter = COCO2YOLO(src_img_dir='/content/BCCD_Dataset/BCCD/JPEGImages', json_file='/content/BCCD_Dataset/BCCD/test.json',
                                 tgt_img_dir='/content/bccd/images/test', tgt_anno_dir='/content/bccd/labels/test')
test_yolo_converter.coco2yolo()

total images 205
total categories 3
total labels 2805
loading image info...
loading done, total images 205
start converting...
converting done, total labels 205
saving txt file...
saving done
total images 87
total categories 3
total labels 1138
loading image info...
loading done, total images 87
start converting...
converting done, total labels 87
saving txt file...
saving done
total images 72
total categories 3
total labels 945
loading image info...
loading done, total images 72
start converting...
converting done, total labels 72
saving txt file...
saving done


In [9]:
!wget -O /content/bccd/bccd.yaml https://raw.githubusercontent.com/chulminkw/DLCV/master/data/util/bccd.yaml

--2021-09-29 01:17:12--  https://raw.githubusercontent.com/chulminkw/DLCV/master/data/util/bccd.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184 [text/plain]
Saving to: ‘/content/bccd/bccd.yaml’

/content/bccd/bccd. 100%[===================>]     184  --.-KB/s    in 0s      

2021-09-29 01:17:12 (9.89 MB/s) - ‘/content/bccd/bccd.yaml’ saved [184/184]



In [10]:
# soft link로 Google Drive Directory 연결. 
!ln -s /content/drive/My\ Drive/ /mydrive
!ls /mydrive
# Google Drive 밑에 Directory 생성. 이미 생성 되어 있을 시 오류 발생. 
!mkdir "/mydrive/ultra_workdir"

 텐서플로로배우는딥러닝  'My Drive'			        ultra_workdir
'2020년 공부.zip'	  object-detection-and-tracking.ipynb   Untitled0.ipynb
'Colab Notebooks'	  pet				        공모전.zip
'Computer Vision'	  test.csv			        공부파일.zip
 MOS.zip		  tf_idf.ipynb
mkdir: cannot create directory ‘/mydrive/ultra_workdir’: File exists


In [15]:
###  10번 미만 epoch는 좋은 성능이 안나옴. 최소 30번 이상 epoch 적용. large 모델 적용 시 batch size가 8보다 클 경우 colab에서 memory 부족 발생.
### 혈소판의 경우 상대적으로 mAP:0.5~0.95 Detection 성능이 좋지 못함. 백혈구 만큼 학습데이터가 적은것도 이유지만, Object 사이즈가 상대적으로 작음.   
!cd /content/yolov5; python train.py --img 640 --batch 8 --epochs 30 --data /content/bccd/bccd.yaml --weights yolov5l.pt \
                                     --project=/mydrive/ultra_workdir --name bccd --exist-ok 

train: weights=yolov5l.pt, cfg=, data=/content/bccd/bccd.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=30, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, entity=None, project=/mydrive/ultra_workdir, name=bccd, exist_ok=True, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-472-gc1bed60 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, tran

In [11]:
# image 파일 inference 
!cd /content/yolov5;python detect.py --source /content/bccd/images/test/BloodImage_00011.jpg \
                            --weights /mydrive/ultra_workdir/bccd/weights/best.pt --conf 0.2 \
                            --project=/content/data/output --name=run_image --exist-ok --line-thickness 2

detect: weights=['/mydrive/ultra_workdir/bccd/weights/best.pt'], source=/content/bccd/images/test/BloodImage_00011.jpg, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/data/output, name=run_image, exist_ok=True, line_thickness=2, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-481-g9988059 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 392 layers, 46611336 parameters, 0 gradients, 114.1 GFLOPs
image 1/1 /content/bccd/images/test/BloodImage_00011.jpg: 480x640 1 WBC, 20 RBCs, 1 Platelets, Done. (0.103s)
Speed: 0.7ms pre-process, 103.3ms inference, 34.4ms NMS per image at shape (1, 3, 640, 640)
Results saved to /content/data/output/run_image


In [13]:
!cd /content/yolov5; python val.py --weights /mydrive/ultra_workdir/bccd/weights/best.pt  --data /content/bccd/bccd.yaml \
                           --project /content/data/output --name=test_result --exist-ok --img 640 --iou 0.65


val: data=/content/bccd/bccd.yaml, weights=['/mydrive/ultra_workdir/bccd/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/data/output, name=test_result, exist_ok=True, half=False
YOLOv5 🚀 v5.0-481-g9988059 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 392 layers, 46611336 parameters, 0 gradients, 114.1 GFLOPs
val: Scanning '/content/bccd/labels/val' images and labels...87 found, 0 missing, 0 empty, 0 corrupted: 100% 87/87 [00:00<00:00, 1132.30it/s]
val: New cache created: /content/bccd/labels/val.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 3/3 [00:10<00:00,  3.66s/it]
                 all         87       1138      0.673      0.829      0.859      0.533
                 WBC         87         87      0.495        